In [12]:
import pygsheets
from oauth2client.service_account import ServiceAccountCredentials
from pathlib import Path
import pandas as pd
pd.set_option('mode.chained_assignment', None)

In [13]:
import pandas as pd
pd.options.display.max_columns = 0

In [14]:

# Load the credentials from the JSON file (replace with the path to your file)
sf_path = r'inductive-gift-355101-48518c54d576.json'
gc = pygsheets.authorize(service_account_file=sf_path)


In [15]:
def get_number_gsheet(service_file_path, spreadsheet_id, sheet_name):
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    wks = sh.worksheet_by_title(sheet_name)
    df = wks.get_as_df(
        numerize=False, value_render='UNFORMATTED_VALUE', empty_value=None)
    df = df.replace(to_replace='', value=None)

    return df

# %%
# lay du lieu tu googhesheet ma khong co du lieu so


def get_df_from_gsheet(service_file_path, spreadsheet_id, sheet_name, start=None):
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    wks = sh.worksheet_by_title(sheet_name)
    df = wks.get_as_df(start, numerize=False, empty_value=None)
    df = df.replace(to_replace='', value=None)
    df.drop_duplicates(inplace=True)

    return df

In [16]:
def get_data(sheet_name, start):
    spreadsheet_id = '1Bmm9GGP3QE5k_T8eODN3Hrh8ICzMgAazYkw0gWNRZDs'

    gc = pygsheets.authorize(service_file=sf_path)
    sh = gc.open_by_key(spreadsheet_id)
    wks = sh.worksheet_by_title(sheet_name)
    df = wks.get_as_df(start, numerize=False, value_render='UNFORMATTED_VALUE', empty_value=None)
    df = df.replace(to_replace='', value=None)
    # df.drop_duplicates(inplace=True)

    df2 = df.transpose()
    df2.iloc[:,1] = df2.iloc[:,1].ffill()
    df3 = df2.transpose()
    # Combine the two lists
    combined_list = [f"{a}|{b}" for a, b in zip(df3.iloc[1].to_list(), df3.iloc[2].to_list())]
    df3.columns = combined_list
    return df3

In [17]:
def extract_daily(df3, sheet_name):
    df_daily = df3.iloc[10:,:]
    df_daily2 = df_daily.melt(id_vars='None|None')
    df_daily2[['user', 'criteria']] =df_daily2['variable'].str.split('|', expand=True)
    df_daily2.rename(columns={'None|None':'report_date'}, inplace=True)
    df_daily2.drop(columns='variable', inplace=True)
    df_daily2.dropna(subset='value', inplace=True)
    df_daily3 = df_daily2.query("value!=0").pivot(columns='criteria', index=['user', 'report_date'], values='value').reset_index()
    df_daily3['report_date'] = df_daily3['report_date'].astype('str') + '/' + sheet_name
    df_daily3['report_date'] = pd.to_datetime(df_daily3['report_date'], format='%d/%m/%Y')
    # df_daily3.to_parquet(rf".\data\daily\{str(sheet_name).replace('/', '-')}.parquet", index=False )
    return df_daily3

In [18]:
def extract_target(df3, sheet_name):
    df_target = df3.iloc[3:10]
    df_target.rename(columns={"None|None":"item"}, inplace=True)
    df_target.query("item.isin(['Target', 'Daily'])", inplace=True)

    df_target2 = df_target.melt(id_vars='item')
    df_target2[['user', 'criteria']] =df_target2['variable'].str.split('|', expand=True)
    # df_target2.rename(columns={'item':'report_date'}, inplace=True)
    df_target2.drop(columns='variable', inplace=True)
    df_target2.dropna(subset='value', inplace=True)

    df_target2.drop_duplicates(subset=['item', 'user'], inplace=True)
    df_target3 = df_target2.query("value!=0").pivot(columns='item', index=['user'], values='value').reset_index()

    df_target3['report_month'] = '01/' + sheet_name
    df_target3['report_month'] = pd.to_datetime(df_target3['report_month'], format='%d/%m/%Y')
    # df_target3.to_parquet(rf".\data\target\{str(sheet_name).replace('/', '-')}.parquet", index=False )
    return df_target3

In [19]:
sheets = [
        '9/2024',
        # '8/2024',
        # '7/2024',
        # '6/2024',
        # '5/2024',
        # '4/2024',
        # '3/2024',
        # '2/2024',
        # '1/2024',
        # '12/2023',
        # '11/2023',
        # '10/2023',
        # '9/2023',
        # '8/2023',
        # '7/2023',
        # '6/2023',
        # '5/2023',
        # '4/2023',
        # '3/2023',
        # '2/2023',
        # '1/2023',
]
for sheet_name in sheets:
    df3 = get_data(sheet_name=sheet_name, start='A1')
    df_daily = extract_daily(df3, sheet_name)
    df_target = extract_target(df3, sheet_name)
    # Get start of month and number of days in the month
    start_of_month = df_target.loc[0,'report_month']
    num_days = start_of_month.days_in_month

    # Create a date range for each day of the month
    date_range = pd.date_range(start_of_month, periods=num_days, freq='D')
    df_target['report_date'] = [date_range]*len(df_target)
    df_target = df_target.explode('report_date')
    df_target['report_day'] = df_target['report_date'].dt.day
    df_target['no_of_day'] = num_days
    df_target['Daily'] = df_target['Daily'].astype(int)

    df_target['Target'] = df_target['Target'].astype(int)
    df_target['daily_target_norm'] =  df_target['Target']/df_target["no_of_day"]
    df_target['mtd_target_norm'] = df_target['daily_target_norm'] * df_target['report_day']
    dta = df_target.merge(df_daily, how='left', on=['user','report_date'])
    dta['Total'] = dta['Total'].fillna(0)
    dta['Total'] = pd.to_numeric(dta['Total'], errors='coerce')
    dta['mtd_actual'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['Total'].cumsum()
    # danh dau ngay dai daily target
    dta['flg_daily']  = 0
    flt = dta['Total'] >= dta['Daily']
    dta.loc[flt, 'flg_daily'] = 1

    # danh dau ngay co tap luyen
    dta['flg_workout'] = 0
    flt = dta['Total'] >0
    dta.loc[flt, 'flg_workout'] = 1


    for col in ['Burpee', 'Core','Pushup', 'Run', 'Squat','flg_daily', 'flg_workout']:
        try:
            dta[col] = pd.to_numeric(dta[col], errors='coerce')
            dta[col] = dta[col].fillna(0)
            new_col = 'mtd_'+col
            dta[new_col] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])[col].cumsum()
        except Exception as e:
            print(e)
    # tinh toan accumulate
    dta.to_parquet(rf".\data\daily\dta_{str(sheet_name).replace('/', '-')}.parquet", index=False )
    print(sheet_name)

9/2024


In [11]:
sheet_name='9/2024'
# df3 = get_data(sheet_name=sheet_name, start='A4')

spreadsheet_id = '1Bmm9GGP3QE5k_T8eODN3Hrh8ICzMgAazYkw0gWNRZDs'

gc = pygsheets.authorize(service_file=sf_path)
sh = gc.open_by_key(spreadsheet_id)
wks = sh.worksheet_by_title(sheet_name)
df = wks.get_as_df(start='A1', numerize=False, value_render='UNFORMATTED_VALUE', empty_value=None)


RefreshError: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})

In [ ]:
df

In [ ]:
df = df.replace(to_replace='', value=None)
# df.drop_duplicates(inplace=True)

df2 = df.transpose()
df2


In [ ]:
df2.iloc[:,1] = df2.iloc[:,1].ffill()
df3 = df2.transpose()
# Combine the two lists
combined_list = [f"{a}|{b}" for a, b in zip(df3.iloc[1].to_list(), df3.iloc[2].to_list())]
df3.columns = combined_list

In [ ]:
df3

In [ ]:
df_daily = extract_daily(df3, sheet_name)
df_target = extract_target(df3, sheet_name)

In [ ]:
df_daily.query("user=='Kiên Trần'")

In [ ]:
start_of_month = df_target.loc[0,'report_month']
num_days = start_of_month.days_in_month

# Create a date range for each day of the month
date_range = pd.date_range(start_of_month, periods=num_days, freq='D')
df_target['report_date'] = [date_range]*len(df_target)
df_target = df_target.explode('report_date')
df_target['report_day'] = df_target['report_date'].dt.day
df_target['no_of_day'] = num_days
df_target['Daily'] = df_target['Daily'].astype(int)

df_target['Target'] = df_target['Target'].astype(int)
df_target['daily_target_norm'] =  df_target['Target']/df_target["no_of_day"]
df_target['mtd_target_norm'] = df_target['daily_target_norm'] * df_target['report_day']


In [ ]:
dta = df_target.merge(df_daily, how='left', on=['user','report_date'])
dta['Total'] = dta['Total'].fillna(0)


In [ ]:
dta

In [ ]:
dta['Total'] = pd.to_numeric(dta['Total'], errors='coerce')

In [ ]:
dta['mtd_actual'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['Total'].cumsum()
# dta.sort_values(by=['user', 'report_date']).groupby(['user'])['Total'].sum()


In [ ]:
dta.query("user=='Kiên Trần'")

In [ ]:
dta['flg_daily']  = 0
flt = dta['Total'] >= dta['Daily']
dta.loc[flt, 'flg_daily'] = 1
dta['mtd_daily'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['flg_daily'].cumsum()


In [ ]:
dta.columns

In [ ]:
for col in ['Burpee', 'Core','Pushup', 'Run', 'Squat']:
    dta[col] = pd.to_numeric(dta[col], errors='coerce')

In [ ]:
dta['mtd_burpee'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['Burpee'].cumsum()
dta['mtd_core'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['Core'].cumsum()
dta['mtd_pushup'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['Pushup'].cumsum()
dta['mtd_run'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['Run'].cumsum()
dta['mtd_squat'] = dta.sort_values(by=['user', 'report_date']).groupby(['user'])['Squat'].cumsum()

In [ ]:
dta.to_parquet(rf".\data\daily\dta_{str(sheet_name).replace('/', '-')}.parquet", index=False )
print(sheet_name)

In [ ]:
df_target

In [ ]:
df_daily['user'].value_counts()

In [ ]:
df_daily.query("user=='An'")

In [ ]:
df= pd.read_parquet('data\daily\dta_8-2024.parquet')

In [ ]:
df

In [ ]:
df.query("report_date=='2023-12-31'")

In [ ]:
import plotly.express as px

In [ ]:
dta_chart = dta.query("report_date=='2024-09-05'")

In [ ]:
dta_chart.info()

In [ ]:
import plotly.graph_objects as go

# Parameters
total_amount = 1000  # Total target
current_value = 1100  # Current value

# Calculate percentage completion
percentage_complete = (current_value / total_amount) * 100

# Create the gauge chart
fig = go.Figure(go.Indicator(
    mode="gauge+number+delta",  # Shows the gauge, number, and delta (difference from total)
    value=current_value,  # Absolute value (current amount)
    number={'valueformat': ',.0f'},  # Display the absolute value without decimal points
    delta={'reference': total_amount, 'position': "top", 'increasing': {'color': "green"}},  # Show difference from total
    gauge={
        'axis': {'range': [0, total_amount]},  # Set the range from 0 to the total
        'bar': {'color': "blue"},  # Color of the gauge bar
        'steps': [
            {'range': [0, total_amount * 0.5], 'color': "#ff4545"},  # Colors for ranges
            {'range': [total_amount * 0.5, total_amount*0.75], 'color': "#fffc45"}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': total_amount  # Mark the total with a threshold line
        }
    }
))

# Add annotation for percentage on a new line
fig.add_annotation(
    x=0.5, y=-0.1,  # Position the annotation
    text=f"({percentage_complete:.0f}%)",  # Display the percentage
    showarrow=False,
    font={'size': 20},
    xanchor="center",
    yanchor="bottom"
)

# Update layout to customize appearance
fig.update_layout(
    title={'text': "Sales Target Progress", 'x': 0.5},  # Title and center it
    height=400
)

# Display the chart
fig.show()


In [ ]:
dta_chart.columns

In [ ]:
import plotly_express as px

In [ ]:
import pandas as pd
import plotly.graph_objects as go


In [ ]:
df= pd.read_parquet('data\daily\dta_9-2024.parquet')
# Pivoting the dataframe
heatmap_data = df.pivot(index='user', columns='report_date', values='flg_workout')
heatmap_data
# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='Greens',
    hoverongaps=False,
    xgap=2,  # Horizontal gap between the cells
    ygap=2,   # Vertical gap between the cells
    # colorscale=[[0, 'white'], [1, 'green']],  # Color scale from white to green
    showscale=False  # Display the color scale
))

# Update layout
fig.update_layout(
    # xaxis_title='Date',
    # yaxis_title='User',
    title='Workout Heatmap',
    # xaxis=dict(showgrid=True),
    # yaxis=dict(showgrid=True)
)

# Display the heatmap
fig.show()


In [ ]:
df= pd.read_parquet('data\daily\dta_8-2024.parquet')


In [ ]:
df.head()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Sample dataframe representing workout data
data = {
    'report_date': pd.date_range(start='2024-09-01', periods=10, freq='D').tolist() * 3,
    'user': ['User A'] * 10 + ['User B'] * 10 + ['User C'] * 10,
    'Total': [5, 7, 6, 9, 4, 8, 7, 10, 5, 0] * 3,  # Adding some 0 values for example
    'burpee': [20, 15, 10, 25, 30, 15, 10, 5, 25, 20] * 3,
    'run': [2.5, 3.0, 1.5, 4.0, 2.0, 3.5, 2.0, 4.5, 3.0, 3.5] * 3
}

df = pd.DataFrame(data)

# Pivot data to get users on y-axis, report_date on x-axis, and Total as values
heatmap_data = df.pivot(index='user', columns='report_date', values='Total')

# Create custom hover text with burpee and run details, formatting the date
hover_text = df.apply(lambda row: f"Date: {row['report_date'].strftime('%Y-%m-%d')}<br>User: {row['user']}<br>Burpee: {row['burpee']}<br>Run: {row['run']}", axis=1).values.reshape(heatmap_data.shape)

# Create text array that replaces 0 with '-'
text_data = heatmap_data.applymap(lambda x: '-' if x == 0 else f"{x}")

# Create the heatmap with Plotly
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='Blues',  # Customize the color scale
    hoverinfo='text',  # Use custom hover info
    hovertext=hover_text,  # Set the custom hover text
    text=text_data.values,  # Show the 'Total' value or '-' in each cell
    texttemplate="%{text}",  # Show the custom text
    textfont={"size": 12},  # Font size of the text in the cells
    xgap=2,  # Add space between cells to simulate a border
    ygap=2   # Add space between cells to simulate a border
))

# Customize the layout, including the black background and thinner margins
fig.update_layout(
    title="Workout Heatmap",
    xaxis_title="Date",
    yaxis_title="User",
    xaxis=dict(showgrid=False),  # Disable gridlines to only show the gap
    yaxis=dict(showgrid=False),  # Disable gridlines to only show the gap
    plot_bgcolor='black',  # Set the plot background color to black (for borders)
    paper_bgcolor='white',  # Keep the paper background white
    margin=dict(l=10, r=10, t=50, b=10),  # Adjust outer margins for thinner outer border
)

# Show the plot
fig.show()


In [ ]:
print(px.colors.sequential.Rainbow_r)